In [ ]:
import requests
import pandas as pd
import json

In [ ]:
url = "https://api.elsevier.com/content/search/scopus/"

In [ ]:
field_table = pd.read_csv('./Abbreviation.csv')

In [ ]:
field_list = field_table['ชื่อย่อ'].to_list()

In [ ]:
eid_field_list = []
sum = 0

In [ ]:
for i in field_list:
    #First Check For Amount
    params = {
        "query": "AFFILORG(Chulalongkorn University) AND PUBYEAR = 2017 AND SUBJAREA ( +" + i.lower() + " )",
        "field": "eid",
        "start": 0
    }
    headers = {
        "Accept": "application/json",
        "X-ELS-APIKey": "fdfa191b9599a981a891f1f6d55bdc95",
    }

    document_count = 0

    response = requests.get(url, headers=headers, params=params)

    if(response.status_code == 200):
        document_count = int(response.json()['search-results']['opensearch:totalResults'])

    sum += document_count

    start_index = 0
    while (start_index < document_count):
        params = {
        "query": "AFFILORG(Chulalongkorn University) AND PUBYEAR = 2017 AND SUBJAREA ( +" + i.lower() + " )",
        "field": "eid",
        "start": start_index
        }

        headers = {
            "Accept": "application/json",
            "X-ELS-APIKey": "fdfa191b9599a981a891f1f6d55bdc95",
        }

        response = requests.get(url, headers=headers, params=params)
        
        if(response.status_code == 200):
            res_json = response.json()
            documents = res_json['search-results']['entry']
            for document in documents:
                print(document['eid'], i)
                eid_field_list.append([document['eid'], i])
    
        start_index += 25
                




In [ ]:
eid_field_list_df = pd.DataFrame(eid_field_list, columns=['Eid', 'Field'])

In [ ]:
with open('./PAPAER_SCRAPING.json') as d:
    data = json.load(d)

In [ ]:
scrape_eid_list = []

for i in data:
    scrape_eid_list.append(i['data']['Eid'])

In [ ]:
eid_field_list_df = pd.DataFrame(eid_field_list, columns=['Eid', 'Field_Abbv'])

In [ ]:
eid_field_list_df = eid_field_list_df[eid_field_list_df['Eid'].isin(scrape_eid_list)]

In [ ]:
field_table = field_table.rename(columns={'ชื่อย่อ' : "Field_Abbv", "ชื่อเต็ม" : "Subject_Field"})

In [ ]:
eid_field_list_df = eid_field_list_df.merge(field_table, how='inner', on='Field_Abbv')

In [ ]:
eid_field_list_df = eid_field_list_df.drop(columns='Field_Abbv')

In [ ]:
eid_field_list_df.to_csv('scrabe_subject_field.csv')